If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.41.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("question_answering_notebook", framework="pytorch")

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](https://github.com/huggingface/notebooks/blob/master/examples/images/question_answering.png?raw=1)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [6]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "sagorsarker/bangla-bert-base"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

**RANDOMIZE**

In [7]:
from datasets import load_dataset, DatasetDict
# Load the dataset
dataset = load_dataset('shakun42/BanglaRQA_to_SquadBn_fact_confirm')

Generating train split:   0%|          | 0/9565 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1182 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

In [8]:
datasets=dataset

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [9]:
filtered_dataset = datasets["validation"].filter(lambda x: all(start != -1 for start in x['answers']['answer_start']))

Filter:   0%|          | 0/1182 [00:00<?, ? examples/s]

In [10]:
len(filtered_dataset)

1182

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [11]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,bn_wiki_0245_05,স্টারশিপ এইচএলএস ইতিহাস,"স্পেসএক্স স্টারশিপের নকশাটি প্রাথমিকভাবে ২০১০-এর দশকের গোড়ার দিকে একটি মহাকাশযান হিসেবে ধারণা করা হয়, যা স্পেসএক্সের প্রধান নির্বাহী এলন মাস্কের পরামর্শে ২০১১ সাল থেকে মঙ্গল গ্রহের মানব বসতি স্থাপনের প্রচেষ্টার জন্য তৈরি করা হয়, প্রথম বসতি স্থাপন ২০২০-এর দশকের মাঝামাঝি সময়ের আগে সম্ভব নয়।\n\nসুযোগ-সুবিধা ২০১৬ সালের মধ্যে কিছুটা বিস্তৃত হয়ে উঠল, যখন এলন মাস্ক বুঝতে পারলেন যে উচ্চ স্তরের নকশা স্পেসএক্স স্টারশিপ যানের জন্য কাজ করছে, যা বৈকল্পিক ভ্রমণের জন্য অনুমোদিত হবে, সেই সাথে সাধারণত গ্রহভ্রমণের জন্য উপযুক্ত হবে এবং বায়ুমণ্ডল বিশিষ্ট ও বায়ুমণ্ডলবিহীন উভয় গ্রহে কাজ করতে পারে। অবশ্য চাঁদের গন্তব্যের উড়ানগুলির জন্য এলন মাস্কের বিশেষ কোন পরামর্শে প্রদান করেননি এবং তিনি বিশেষভাবে বলেছিলেন যে চাঁদের গন্তব্য মঙ্গল গ্রহের পথে প্রয়োজনীয় পদক্ষেপ ছিল না।",স্পেসএক্সের প্রধান নির্বাহী এলন মাস্ক কবে মঙ্গল গ্রহে যেতে চান?,"{'answer_start': [], 'text': []}"
1,bn_wiki_2326_03,কারাল সভ্যতা,"কারাল সভ্যতা বা কারাল-সুপে সভ্যতা এখনও পর্যন্ত জানা সবচেয়ে প্রাচীন আন্দীয় সভ্যতা। দক্ষিণ আমেরিকার সুপ্রাচীন এই সভ্যতা বহু ক্ষেত্রে নর্তে চিকো সভ্যতা নামেও পরিচিত।[পা ১] প্রথম নামটি এসেছে পেরুর সুপে উপত্যকায় অবস্থিত কারাল অঞ্চলের নাম থেকে। এইস্থানেই এই সভ্যতার সবচেয়ে বড় ও গুরুত্বপূর্ণ স্তূপটি আবিস্কৃত হয়েছে। তাছাড়া এই অঞ্চলটি, যতদূর বোঝা গেছে, এই সভ্যতায় একটি অত্যন্ত পবিত্র স্থান বলেও বিবেচিত হত। অন্যদিকে পেরুর এই অঞ্চলকে কথ্য ভাষায় বর্তমানে নর্তে চিকো (স্পেনীয়, অর্থ উত্তরের ছোট্ট স্থান) বলা হয়। তার থেকেই এই দ্বিতীয় নামটির সৃষ্টি। খ্রিস্টজন্মের ৯০০০ বছর আগেই এই সভ্যতার সূচনা হয়। তবে খ্রিস্টপূর্ব ৩০০০ - ১৮০০ অব্দকে এই সভ্যতার সবচেয়ে বেশি বিকাশের সময় বলে মনে করা হয়। উত্তর-মধ্য পেরুর সমুদ্র উপকূলে এই সভ্যতার অন্তত ৩০টি কেন্দ্র খুঁজে পাওয়া গেছে। এদের মধ্যে কারাল, আসপেরো, উয়ারিকাঙ্গা, কাবালেত, প্রভৃতি স্থলে খননকার্যের মাধ্যমে এই সভ্যতার প্রচূর নিদর্শন খুঁজে পাওয়া গেছে। এদের মধ্যে রয়েছে পাথরে তৈরি সম্ভাব্য বড় বড় মন্দিরের উঁচু প্ল্যাটফর্ম, বসবাসের জন্য তৈরি বাড়ির ধ্বংসস্তূপ, বেশ কিছু ঢিবি, প্ল্যাটফর্মের উপর খাওয়াদাওয়ার চিহ্ন, হাড়ের তৈরি বেশ কিছু বাঁশি, প্রভৃতি। তবে নব্যপ্রস্তর যুগের এই সভ্যতায় ধাতুর ব্যবহার জানা ছিল না। এমনকী মৃৎপাত্র তৈরি বা ব্যবহারের কোনও নিদর্শনও এখানে খুঁজে পাওয়া যায়নি। কিন্তু প্রত্নতাত্ত্বিক খননকার্য থেকে এখানে যথেষ্ট জটিল একটি রাজনৈতিক ও সামাজিক ক্রিয়াকলাপের অস্তিত্ব পরিষ্কার বোঝা যায়। কালের বিচারে এই সভ্যতার সর্বোত্তম বিকাশের সময়টি ছিল পুরনো পৃথিবীর সুমের সভ্যতার থেকে হাজার বছর পরে, কিন্তু মিশরে যে সময়ে পিরামিডগুলি নির্মাণ হয়, তার সমসাময়িক। পশ্চিম গোলার্ধের অপর প্রাচীন সভ্যতা কেন্দ্র মেসোআমেরিকার থেকে এই সভ্যতা অন্তত ২০০০ বছর প্রাচীন।",কত বছর আগে কারাল সভ্যতার সূচনা হয়?,"{'answer_start': [548, 548], 'text': ['খ্রিস্টজন্মের ৯০০০', 'খ্রিস্টজন্মের ৯০০০']}"
2,bn_wiki_0389_04,টায়ার ব্যবহার,"টায়ার মুলত যানবাহনের ধরন অনুযায়ী বিভিক্ত। এটা বহন করা লোড এবং তাদের অ্যাপ্লিকেশনের দ্বারা পৃথক হতে পারে, যেমন। একটি মোটর গাড়ির, বিমান বা সাইকেল।\nহালকা-মাঝারি ক্ষমতাসম্পন্ন\nযাত্রী গাড়ির জন্য হালকা-মাঝারি ​​টায়ার ড্রাইভ চাকাতে ৫৫০ থেকে ১,১০০ পাউন্ড (২৫০ থেকে ৫০০ কেজি) পর্যন্ত লোড বহন করে। লাইট-টু-মিডিয়াম ডিউটি ​​ট্রাক এবং ভ্যানগুলি ড্রাইভ চাকাতে ১,১০০ থেকে ৩,৩০০ পাউন্ড (৫০০ থেকে ১,৫০০ কেজি) পর্যন্ত লোড বহন করে। \nভারী ক্ষমতাসম্পন্ন\nবড় ট্রাক এবং বাসের জন্য ভারী ক্ষমতাসম্পন্ন টায়ারগুলি বিভিন্ন প্রকারের প্রোফাইলে আসে এবং ড্রাইভ চাকাতে ৪,০০০ থেকে ৫,৫০০ পাউন্ড (১,৮০০ থেকে ২৫০০ কেজি) পর্যন্ত লোড বহন করে। এইগুলি সাধারণত ড্রাইভ অ্যাক্সলে ট্যান্ডেমে মাউন্ট করা হয়।\nঅন্যান্য\nবিমান, সাইকেল এবং শিল্প অ্যাপ্লিকেশন বিভিন্ন স্বতন্ত্র নকশা প্রয়োজনীয়তা আছে।\nসাইকেল- সাইকেল টায়ারগুলি রাস্তায় বা অনির্বাচিত উপকূলে চলাচলের জন্য ডিজাইন করা যেতে পারে এবং দুইটি চাকার সাথে গাড়িগুলিতে মাউন্ট করা যেতে পারে। তিনটি প্রধান ধরনের: ক্লিনিক, তারযুক্ত এবং নলাকার।\nমোটর সাইকেল- মোটরসাইকেল টায়ার ট্র্যাকশন প্রদান করে, পরিধান প্রতিরোধ করে, পৃষ্ঠের অনিয়মগুলি শোষণ করে এবং মোটরসাইকেলটিকে কাউন্টারস্টিয়ারিংয়ের মাধ্যমে ঘুরিয়ে দেয়। মাটির সাথে দুটি টায়ারের যোগাযোগ নিরাপত্তা, ব্রেকিং, জ্বালানি অর্থনীতি, গোলমাল এবং রাইডারের আরামকে প্রভাবিত করে।\nব্যবহার সমাপ্ত

## Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [14]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [15]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 100, 100, 100, 100, 1045, 102, 100, 100, 100, 100, 1013, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [16]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [17]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Without any truncation, we get the following length for the input IDs:

In [18]:
len(tokenizer(example["question"], example["context"])["input_ids"])

542

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [19]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [20]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of `input_ids`, but several:

In [21]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 321]

And if we decode them, we can see the overlap:

In [22]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

2024-06-27 14:35:52.301493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 14:35:52.301602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 14:35:52.442819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[CLS] কত সালে ঢাকা আলিযা মাদরাসা ঢাকায সথানানতরের পরবে বাংলাদেশ ও ভারতের ফাজিল পরীকষা কলকাতা আলিযা মাদরাসার অধীনে অনষঠিত হতো? [SEP] ফাজিল পরীকষা বাংলাদেশ ও ভারতের আলিযা মাদরাসায অনষঠিত একটি সরকারি পরীকষা । ফাজিল পরীকষা বাংলাদেশে ডিগরি সমমানের, কখনো সনাতক সমমানের একটি পরীকষা, যা একটি ফাজিল মাদরাসায অনষঠিত হযে থাকে । তবে ভারতে ফাজিল পরীকষাকে উচচ মাধযমিক শরেণীর ( [UNK] বা [UNK] কলাস ) মান বলে বিবেচিত করা হয । ফাজিল পরীকষা বাংলাদেশ ভারত ও পাকিসতানের সরকারি সবীকত আলিযা মাদরাসায পরচলিত রযেছে । বাংলাদেশের ফাজিল পরীকষা ইসলামি আরবি বিশববিদযালযের অধীনে অনষঠিত হযে থাকে ও ভারতের ফাজিল পরীকষা পশচিমবঙগ মাদরাসা শিকষা পরষদের অধীনে অনষঠিত হযে থাকে । [UNK] সালে ঢাকা আলিযা মাদরাসা ঢাকায সথানানতরের পরবে বাংলাদেশ ও ভারতের ফাজিল পরীকষা কলকাতা আলিযা মাদরাসার অধীনে অনষঠিত হতো । ফাযিল পরীকষা বরতমানে ইসলামি আরবী বিশববিদযালযের অধীনে অনষঠিত হয । যা পরবে মাদরাসা বোরড ও ইসলামি বিশববিদযালযের আধীনে অনষঠিত হত । মাদরাসা - ই - আলিযা ঢাকায সথানানতরিত হলে [UNK] সালে মাদরাসা বোরডের ফাজিলগলো পরীকষা ঢাকা বিশববিদযালয কর

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [23]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 2), (3, 7), (8, 12), (13, 16), (16, 19), (20, 28), (29, 33), (33, 34), (36, 37), (38, 42), (42, 43), (44, 46), (46, 48), (49, 55), (56, 64), (65, 66), (67, 73), (74, 79), (80, 83), (83, 86), (86, 87), (88, 94), (95, 98), (98, 101), (102, 111), (112, 117), (118, 120), (121, 122), (123, 126), (127, 130), (129, 130), (131, 132), (0, 0), (0, 5), (6, 9), (9, 12), (12, 13), (14, 22), (23, 24), (25, 31), (32, 35), (35, 38), (39, 47), (47, 48), (50, 52), (53, 54), (55, 58), (59, 63), (64, 70), (71, 74), (74, 77), (77, 78), (78, 79), (80, 85), (86, 89), (89, 92), (92, 93), (94, 103), (104, 107), (108, 110), (111, 118), (118, 119), (120, 123), (123, 124), (123, 124), (125, 128), (128, 130), (130, 131), (132, 139), (140, 144), (145, 148), (148, 151), (151, 152), (152, 153), (154, 156), (157, 161), (162, 167), (168, 176), (176, 177), (179, 181), (182, 183), (184, 187), (188, 190), (191, 192), (193, 197), (197, 198), (199, 202), (203, 208), (209, 214), (215, 218), (218, 221), (221, 224

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [24]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

কত কত


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [25]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [26]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

184 184


And we can double check that it is indeed the theoretical answer:

In [27]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

[UNK]
১৯৪৭


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [28]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [29]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [30]:
features = prepare_train_features(datasets['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [31]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Map:   0%|          | 0/9565 [00:00<?, ? examples/s]

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Map:   0%|          | 0/1172 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [32]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.


To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [34]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
#     f"{model_name}-finetuned-squad1.2",
    f"{model_name}-finetuned-brqa-confirmation",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="no",
    push_to_hub=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-squad"` or `"huggingface/bert-finetuned-squad"`).

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [35]:
from transformers import default_data_collator

data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [36]:
# small_train_dataset = tokenized_datasets["train"].select(range(80000))  # Selecting the first 1000 examples

# Now, you can pass this smaller dataset to the Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

We can now finetune our model by just calling the `train` method:

In [37]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.717600,1.499132
2,1.371900,1.345840
3,1.073000,1.411910


TrainOutput(global_step=3405, training_loss=1.4382369031640052, metrics={'train_runtime': 2400.2058, 'train_samples_per_second': 22.696, 'train_steps_per_second': 1.419, 'total_flos': 1.0675409644827648e+16, 'train_loss': 1.4382369031640052, 'epoch': 3.0})

Since this training is particularly long, let's save the model just in case we need to restart.

In [38]:
# trainer.save_model("test-brqa-squad-augmented-trained-indic")
# trainer.save_model("test-BanglaRQA-squad-trained-indic")

## Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [39]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions, let's have a look a the logits:

In [40]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

We have one logit for each feature and each token. The most obvious thing to predict an answer for each featyre is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

In [41]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 96,   0, 125,   0, 111,   0,  13,   0,   0,   0,  24,   0,  20,   0,
           0,   0], device='cuda:0'),
 tensor([103,   0,   0,   0,   0,   0,   0,   0,   0,   0,  34,   0,  25,   0,
           0,   0], device='cuda:0'))

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.


To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [42]:
n_best_size = 10

In [43]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [44]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [45]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [46]:
raw_predictions = trainer.predict(validation_features)

The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [47]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [48]:
max_answer_length = 15

In [49]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 15.0645275, 'text': 'ল্যুভর জাদুঘরে দুটি'},
 {'score': 15.060902, 'text': 'ল্যুভর'},
 {'score': 14.784924, 'text': 'ল্যুভর জা'},
 {'score': 13.469997, 'text': 'দুটি'},
 {'score': 13.227232, 'text': 'ল্যুভর জাদুঘরে'},
 {'score': 12.536023, 'text': 'ল্যুভর জাদুঘরে দুটি সারগনিক'},
 {'score': 11.953453, 'text': 'ল্যুভর জাদ'},
 {'score': 11.604195, 'text': 'ল্যুভর জাদুঘরে দুটি সারগন'},
 {'score': 11.369153, 'text': 'ল্যুভর জাদুঘরে দুটি সারগনিক বিজ'},
 {'score': 10.9898205, 'text': 'ল্যুভর জাদুঘরে দ'}]

We can compare to the actual ground-truth answer:

In [50]:
datasets["validation"][0]["answers"]

{'answer_start': [292, 292], 'text': ['ল্যুভর জাদুঘরে', 'ল্যুভর জাদুঘরে']}

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [51]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [52]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [53]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 1182 example predictions split into 2279 features.


  0%|          | 0/1182 [00:00<?, ?it/s]

Then we can load the metric from the datasets library.

In [54]:

!pip install evaluate -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:
import evaluate

metric = evaluate.load("squad_v2")

In [56]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]
]

In [57]:
 formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]


In [58]:
for pred, true in zip(formatted_predictions, theoretical_answers):
    print("ID:", pred["id"])
    print("Predicted Answer:", pred["prediction_text"])
    print("True Answer:", true["answers"])
    print()

ID: bn_wiki_2812_01
Predicted Answer: ল্যুভর জাদুঘরে দুটি
True Answer: {'answer_start': [292, 292], 'text': ['ল্যুভর জাদুঘরে', 'ল্যুভর জাদুঘরে']}

ID: bn_wiki_2812_02
Predicted Answer: সুসা (যেখানে এগুলি
True Answer: {'answer_start': [368, 368], 'text': ['খুব সম্ভবত মেসোপটেমিয়া ত্থেকে', 'খুব সম্ভবত মেসোপটেমিয়া ত্থেকে']}

ID: bn_wiki_2812_03
Predicted Answer: ল্যুভর জাদুঘরে দুটি সারগনিক বিজয় ফলক এর অংশবিশেষ আছে
True Answer: {'answer_start': [399, 399], 'text': ['দ্বাদশ শতাব্দীতে', 'দ্বাদশ শতাব্দীতে']}

ID: bn_wiki_2812_04
Predicted Answer: সেমেটিক ভাষাকে
True Answer: {'answer_start': [484, 484], 'text': ['সেমেটিক (আক্কাদীয়ান)', 'সেমেটিক (আক্কাদীয়ান)']}

ID: bn_wiki_2812_05
Predicted Answer: রাজা, এনলিলের রাজ্যপাল[এনসি
True Answer: {'answer_start': [], 'text': []}

ID: bn_wiki_2275_01
Predicted Answer: সেপ্টেম্বর ৩, ২১১২ সালে,  মাৎসুশিবা
True Answer: {'answer_start': [26, 26], 'text': ['মাৎসুশিবা রোবোট', 'মাৎসুশিবা রোবোট']}

ID: bn_wiki_2275_02
Predicted Answer: সেপ্টেম্বর ৩, ২১১২
T

In [61]:
theoretical_answers[2]


{'id': 'bn_wiki_2812_03',
 'answers': {'answer_start': [399, 399],
  'text': ['দ্বাদশ শতাব্দীতে', 'দ্বাদশ শতাব্দীতে']}}

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [62]:
metric.compute(predictions=formatted_predictions, references=theoretical_answers)

{'exact': 19.712351945854483,
 'f1': 30.779275682002332,
 'total': 1182,
 'HasAns_exact': 26.84331797235023,
 'HasAns_f1': 41.9137141199617,
 'HasAns_total': 868,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 314,
 'best_exact': 26.6497461928934,
 'best_exact_thresh': 0.0,
 'best_f1': 32.629952500953316,
 'best_f1_thresh': 0.0}

In [63]:
context = 'পৃথিবী বাঁচার সম্পর্কে জানা গেছে যে জলবায়ু পরিবর্তনের কারণে একটি প্রধান সংকেত বৃদ্ধি পাবে বিপ্লবী পরিবেশ। জলবায়ু পরিবর্তনের মোট প্রভাব ও প্রতিক্রিয়া এখনো সম্পূর্ণ জানা নেই। তবে, এটির সাথে সম্পর্কিত অনেক কারণ ও কারণ সমূহ অবশ্যই বিবেচনা করা উচিত। এটি মানুষের জীবনের অনেক দিক প্রভাবিত করতে পারে, যেমন আপেক্ষিক তাপমাত্রা, সমুদ্র স্তর, ও জলপাই ঘাতক প্রাকৃতিক ঘটনা সহ।'
question='কোন উপায়ে মানুষের কার্বন নির্গত অংশ কমিয়ে তুলতে পারে এবং পরিবেশের জলবায়ু পরিবর্তনের প্রতিরোধ করতে সাহায্য করতে পারে?'

In [64]:
import torch

# Tokenize inputs
tokenized_input = tokenizer.encode_plus(context, question, return_tensors="pt")

# Print tokenized input
print("Tokenized Input:", tokenized_input)

# Model Inference
with torch.no_grad():
    # Move input tensors to the same device as the model
    tokenized_input = {key: value.to(model.device) for key, value in tokenized_input.items()}
    # Forward pass
    outputs = model(**tokenized_input)
    start_logits, end_logits = outputs.start_logits, outputs.end_logits

# Decode Answers
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)
answer_span = tokenized_input['input_ids'][0][start_index : end_index + 1]

# Post-processing
answer = tokenizer.decode(answer_span)

print("Predicted Answer:", answer)
print("Start Logits:", start_logits)
print("End Logits:", end_logits)


Tokenized Input: {'input_ids': tensor([[  101,  2415, 21701,  2765, 37053,  2046,  7505, 12041,  2071,  2295,
          2149,  2060,  4368,  4393,  9294, 96754,  2040,  2147,  2051,  2076,
         21992,  6311, 26535, 19349,  3428, 37320, 44929,  2765,  2679,  1014,
          4368,  4393,  9294, 96754,  2040,  2446, 21327,  2076, 14504,  2038,
         50569,  3924,  5561,  7724,  2112,  2094,  2395,  7505, 12041,  2772,
          2295,  2128,  1014,  2087,  1011,  5182,  2179,  7505, 12041, 59134,
          2115,  2164,  2038,  2164,  7505,  5694, 26465,  9294,  2062,  3388,
          2047,  2351,  1014,  2135,  3175, 47678,  2960,  2115,  2753,  2076,
         53609,  3283,  2059,  2083,  1011,  2292, 67162, 77260,  3924,  8230,
         32641,  2429,  1011,  7505, 23054, 36079,  2046,  1011,  2038, 11012,
         14018,  6407, 81528,  3012,  2367,  3169,  1014,   102,  2285,  5803,
         43006,  8844,  3175, 47678,  3697, 12532, 21962,  4359,  2202, 46972,
          8844, 14973

In [65]:
print(answer)

[CLS]


You can now upload the result of the training to the Hub, just execute this instruction:

In [66]:
trainer.push_to_hub()

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


model.safetensors:   0%|          | 0.00/655M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1719499071.2eef236a7ec0.34.0:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shakun42/bangla-bert-base-finetuned-brqa-confirmation/commit/6a5ae09e0ad150f1c5f398e38dd00166dab4387f', commit_message='End of training', commit_description='', oid='6a5ae09e0ad150f1c5f398e38dd00166dab4387f', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("sgugger/my-awesome-model")
```

In [67]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "shakun42/bangla-bert-base-finetuned-brqa-confirmation"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)


config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/655M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.86M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [68]:
qa_input = {
    'question': 'লালন ফকির কোন সঙ্গীতধারার প্রধান ব্যক্তিত্ব?',
    'context': 'বাংলায় বাউল সংগীত একটি জনপ্রিয় সঙ্গীতধারা। লালন ফকির বাউল গানের একজন প্রধান ব্যক্তিত্ব। তার গানগুলির মধ্যে দার্শনিক ও আধ্যাত্মিক বিষয়বস্তু আছে।'
}
result = nlp(qa_input)
print(result)

{'score': 0.03435133770108223, 'start': 55, 'end': 65, 'answer': 'বাউল গানের'}
